In [366]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

In [367]:
test_pred = pd.read_csv("test_pred_tensor_flow.csv",encoding = "ISO-8859-1")

In [368]:
train_df=pd.read_csv('split_train_pred_tensor_flow.csv')
test_df=pd.read_csv('test_limpio_sin_vect_textos.csv')
sample = pd.read_csv("sample_submission.csv",encoding = "ISO-8859-1")

In [369]:
test_df['pred_tf'] = test_pred.tensor_flow_pred

In [370]:
#Agrego las otras predicciones como features
pred_test_catboost=pd.read_csv('feature_pred/prediccion_test_catBoost.csv')
pred_test_bayes=pd.read_csv('feature_pred/prediccion_test_bayes.csv')
pred_test_svm=pd.read_csv('feature_pred/prediccion_test_SVM.csv')

In [371]:
test_df['pred_catboost'] = pred_test_catboost.target
test_df['pred_bayes'] = pred_test_bayes.target
test_df['pred_svm'] = pred_test_svm.target

In [372]:
def redond(x):
    result=0
    if(x>0.5):
        result=1
    return(result)

In [373]:
train_df['pred_tf_redond'] = train_df['tensor_flow_pred'].apply(lambda x: redond(x))

In [374]:
f1_score(train_df.target, train_df['pred_tf_redond'], average='macro')

0.8108347960707782

# Veo la distribucion de valores mal predichos

In [375]:
def redond_decim(x):
    result=1
    if(x<0.1):
        result=0.1
    elif(x<0.2):
        result=0.2
    elif(x<0.3):
        result=0.3
    elif(x<0.4):
        result=0.4
    elif(x<0.5):
        result=0.5
    elif(x<0.6):
        result=0.6
    elif(x<0.7):
        result=0.7
    elif(x<0.8):
        result=0.8
    elif(x<0.9):
        result=0.9
    return(result)

In [376]:
train_df['pred_tf_decim'] = train_df['tensor_flow_pred'].apply(lambda x: redond_decim(x))

# Cuento la cantidad de elementos predichos correctamente por TF segun su porcentaje

In [377]:
valores_mal_predichos_tensor = train_df[train_df['pred_tf_redond']==train_df.target]
valores_mal_predichos_tensor.shape

(1237, 2844)

In [378]:
dis_tf = valores_mal_predichos_tensor.groupby('pred_tf_decim').agg({'pred_tf_redond':'count'})
dis_tf

,pred_tf_redond
pred_tf_decim,
0.1,239
0.2,234
0.3,127
0.4,67
0.5,28
0.6,40
0.7,38
0.8,33
0.9,61


# Cuento la cantidad de elementos predichos correctamente por CatBoost segun su porcentaje

In [379]:
valores_mal_predichos_catboost = train_df[train_df['pred_catbooost']==train_df.target]
valores_mal_predichos_catboost.shape

(1169, 2844)

In [380]:
dis_cb = valores_mal_predichos_catboost.groupby('pred_tf_decim').agg({'pred_catbooost':'count'})
dis_cb

,pred_catbooost
pred_tf_decim,
0.1,231
0.2,225
0.3,122
0.4,62
0.5,27
0.6,47
0.7,32
0.8,31
0.9,52


# Cuento la cantidad de elementos predichos correctamente por bayes segun su porcentaje

In [381]:
valores_mal_predichos_bayes = train_df[train_df['pred_bayes']==train_df.target]
valores_mal_predichos_bayes.shape

(1205, 2844)

In [382]:
dis_bay = valores_mal_predichos_bayes.groupby('pred_tf_decim').agg({'pred_bayes':'count'})
dis_bay

,pred_bayes
pred_tf_decim,
0.1,237
0.2,230
0.3,126
0.4,60
0.5,33
0.6,48
0.7,33
0.8,33
0.9,53


# Cuento la cantidad de elementos predichos correctamente por SVM segun su porcentaje

In [383]:
valores_mal_predichos_svm = train_df[train_df['pred_svm']==train_df.target]
valores_mal_predichos_svm.shape

(1202, 2844)

In [384]:
dis_svm = valores_mal_predichos_bayes.groupby('pred_tf_decim').agg({'pred_svm':'count'})
dis_svm

,pred_svm
pred_tf_decim,
0.1,237
0.2,230
0.3,126
0.4,60
0.5,33
0.6,48
0.7,33
0.8,33
0.9,53


# Cuento la cantidad de elementos predichos correctamente por el promedio de las tf,catboost, y svm segun su porcentaje

In [385]:
train_df['ens_tf_cb_svm'] = (train_df['pred_tf_redond']+train_df['pred_catbooost']+train_df['pred_svm'])/3
train_df['ens_tf_cb_svm'] = train_df['ens_tf_cb_svm'].apply(lambda x: redond(x))

In [386]:
valores_mal_predichos_ens = train_df[train_df['ens_tf_cb_svm']==train_df.target]
valores_mal_predichos_ens.shape

(1239, 2845)

In [387]:
dis_ens_tf_cb_svm = valores_mal_predichos_ens.groupby('pred_tf_decim').agg({'ens_tf_cb_svm':'count'})
dis_ens_tf_cb_svm

,ens_tf_cb_svm
pred_tf_decim,
0.1,238
0.2,234
0.3,130
0.4,67
0.5,31
0.6,49
0.7,33
0.8,35
0.9,54


# Cuento la cantidad de elementos predichos correctamente por el promedio de las tf,bayes, y svm segun su porcentaje

In [388]:
train_df['ens_tf_bayes_svm'] = (train_df['pred_tf_redond']+train_df['pred_bayes']+train_df['pred_svm'])/3
train_df['ens_tf_bayes_svm'] = train_df['ens_tf_bayes_svm'].apply(lambda x: redond(x))

In [389]:
valores_mal_predichos_ens = train_df[train_df['ens_tf_bayes_svm']==train_df.target]
valores_mal_predichos_ens.shape

(1228, 2846)

In [391]:
dis_ens_tf_bayes_svm = valores_mal_predichos_ens.groupby('pred_tf_decim').agg({'ens_tf_bayes_svm':'count'})
dis_ens_tf_bayes_svm

,ens_tf_bayes_svm
pred_tf_decim,
0.1,238
0.2,232
0.3,129
0.4,62
0.5,34
0.6,47
0.7,32
0.8,32
0.9,55


# Cuento la cantidad de elementos predichos correctamente por el promedio de las tf,cb, y bayes segun su porcentaje

In [392]:
train_df['ens_tf_bayes_cb'] = (train_df['pred_tf_redond']+train_df['pred_bayes']+train_df['pred_catbooost'])/3
train_df['ens_tf_bayes_cb'] = train_df['ens_tf_bayes_cb'].apply(lambda x: redond(x))

In [393]:
valores_mal_predichos_ens = train_df[train_df['ens_tf_bayes_cb']==train_df.target]
valores_mal_predichos_ens.shape

(1237, 2847)

In [394]:
dis_ens_tf_bayes_cb = valores_mal_predichos_ens.groupby('pred_tf_decim').agg({'ens_tf_bayes_cb':'count'})
dis_ens_tf_bayes_cb

,ens_tf_bayes_cb
pred_tf_decim,
0.1,238
0.2,234
0.3,130
0.4,68
0.5,29
0.6,48
0.7,34
0.8,32
0.9,58


# Cuento la cantidad de elementos predichos correctamente por el promedio de las svm,cb, y bayes segun su porcentaje

In [395]:
train_df['ens_svm_bayes_cb'] = (train_df['pred_svm']+train_df['pred_bayes']+train_df['pred_catbooost'])/3
train_df['ens_svm_bayes_cb'] = train_df['ens_svm_bayes_cb'].apply(lambda x: redond(x))

In [396]:
valores_mal_predichos_ens = train_df[train_df['ens_svm_bayes_cb']==train_df.target]
valores_mal_predichos_ens.shape

(1218, 2848)

In [397]:
dis_ens_svm_bayes_cb = valores_mal_predichos_ens.groupby('pred_tf_decim').agg({'ens_svm_bayes_cb':'count'})
dis_ens_svm_bayes_cb

,ens_svm_bayes_cb
pred_tf_decim,
0.1,238
0.2,232
0.3,129
0.4,63
0.5,36
0.6,48
0.7,31
0.8,37
0.9,51


# Junto todos

In [398]:
dis_tf['pred_cb'] = dis_cb['pred_catbooost']
dis_tf['pred_bayes'] = dis_bay['pred_bayes']
dis_tf['pred_svm'] = dis_svm['pred_svm']
dis_tf['pred_ens_tf_cb_svm'] = dis_ens_tf_cb_svm['ens_tf_cb_svm']
dis_tf['ens_tf_bayes_svm'] = dis_ens_tf_bayes_svm['ens_tf_bayes_svm']
dis_tf['ens_tf_bayes_cb'] = dis_ens_tf_bayes_cb['ens_tf_bayes_cb']
dis_tf['ens_svm_bayes_cb'] = dis_ens_svm_bayes_cb['ens_svm_bayes_cb']
dis_tf

,pred_tf_redond,pred_cb,pred_bayes,pred_svm,pred_ens_tf_cb_svm,ens_tf_bayes_svm,ens_tf_bayes_cb,ens_svm_bayes_cb
pred_tf_decim,,,,,,,,
0.1,239,231,237,237,238,238,238,238
0.2,234,225,230,230,234,232,234,232
0.3,127,122,126,126,130,129,130,129
0.4,67,62,60,60,67,62,68,63
0.5,28,27,33,33,31,34,29,36
0.6,40,47,48,48,49,47,48,48
0.7,38,32,33,33,33,32,34,31
0.8,33,31,33,33,35,32,32,37
0.9,61,52,53,53,54,55,58,51


# La moraleja de esto: En los valores cuya prediccion de tensor flow esta entre 0.4<x<=0.6, me conviene hacer un ensamble, lo pruebo con test y lo subo

# Verifico con el train:

In [399]:
train_df['ens'] = (train_df['pred_tf_redond']+train_df['pred_catbooost']+train_df['pred_svm'])/3

In [400]:
train_df['new_target'] = np.where(((train_df['pred_tf_decim']>0.4) & (train_df['pred_tf_decim']<=0.6)), train_df['ens'], train_df['pred_tf_decim'])

In [401]:
train_df['new_target'] = train_df['new_target'].apply(lambda x: redond(x))

In [402]:
f1_score(train_df.target, train_df['new_target'], average='macro')

0.8181149628158811

In [403]:
test_df['ens'] = (test_df['pred_tf']+test_df['pred_catboost']+test_df['pred_svm'])/3

In [404]:
test_df['target'] = np.where(((test_df['pred_tf']>0.4) & (test_df['pred_tf']<=0.6)), test_df['ens'], test_df['pred_tf'])

In [405]:
test_df['target'] = test_df['target'].apply(lambda x: redond(x))

In [406]:
sample.target = test_df.target

In [407]:
sample.target.value_counts()

0    2094
1    1169
Name: target, dtype: int64

In [408]:
sample.to_csv('neural_network_with_ens.csv', index=False)